## Image example

In [71]:
from PIL import Image
import numpy as np
import torch
# 000000093965# Load and preprocess the image
image = Image.open("/home/bibhabasum/projects/IIIT/data/coco/val2017/000000093965.jpg")
iwidth, iheight = image.size
image = image.resize((640, 640))  # Resize to model input size
image = np.asarray(image).astype(np.float32) / 255.0  # Normalize to [0, 1]
image = torch.tensor(image).permute(2, 0, 1).unsqueeze(0)  # Convert to tensor and add batch dimension
# image = torch.concat([image, image],axis=0)

In [ ]:
from utils.model import DetectionModel

model = DetectionModel(nc = 80)
    # print(model)
weights = torch.load('/home/bibhabasum/projects/IIIT/ultralytics/model_state_dict.pth',map_location="cpu") # Load weights
model.load_state_dict(weights, strict=True)  # Load model weights

model.eval()

output = model(image) 

/tmp/ipykernel_3434970/2002377975.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load('/home/bibhabasum/projects/IIIT/ultralytics/model_state_dict.pth',

In [73]:
output.shape

torch.Size([1, 300, 6])

In [74]:
from utils import non_max_suppression

final = non_max_suppression(output,0.15,0.70)

In [75]:
final[0]

tensor([[ 55.5909, 137.8917, 430.5930, 598.5034,   0.8857,  22.0000],
        [420.7444,  60.7789, 550.5896, 307.1403,   0.8780,  22.0000]],
       grad_fn=<IndexBackward0>)

In [ ]:
# x1, y1, x2, y2, score, cls_id = [ 55.5909, 137.8917, 430.5930, 598.5034,   0.8857,  22.0000]
# width = x2 - x1
# height = y2 - y1
# # Scale bbox back to original image dimensions
# orig_width, orig_height = iwidth, iheight
# scale_x = orig_width / 640
# scale_y = orig_height / 640

# x1 = x1 * scale_x
# y1 = y1 * scale_y
# width = width * scale_x
# height = height * scale_y
# [](55.5909, 98.46329203124999, 375.00210000000004, 328.90554203125004)

In [63]:
import cv2

from utils.classes_dict import classes_dict
from IPython.display import display, Image as IPImage

# Convert the image tensor to a numpy array and scale it back to [0, 255]
image_np = (image.squeeze().permute(1, 2, 0).numpy() * 255).astype(np.uint8)

# Draw bounding boxes
for detection in final[0]:  # Iterate over detections
    x1, y1, x2, y2, conf, cls = detection.tolist()
    class_name = classes_dict.get(int(cls), "Unknown")
    label = f"{class_name} {conf:.2f}"
    
    # Draw rectangle
    cv2.rectangle(image_np, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
    # Put label
    cv2.putText(image_np, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image (if running locally)
# display(IPImage("temp_output.jpg"))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Save the image with bounding boxes
cv2.imwrite("output_with_bboxes.jpg", image_np)

True

In [67]:
import cv2

from utils.classes_dict import classes_dict
from IPython.display import display, Image as IPImage

# Convert the image tensor to a numpy array and scale it back to [0, 255]
image_np = (image.squeeze().permute(1, 2, 0).numpy() * 255).astype(np.uint8)
d= [[421.06, 44.31, 551.61, 218.84,0.90,81],[57.38, 94.27, 430.36, 423.19,0.90,81]]
# Draw bounding boxes
for detection in d:  # Iterate over detections
    x1, y1, x2, y2, conf, cls = detection
    class_name = classes_dict.get(int(cls), "Unknown")
    label = f"{class_name} {conf:.2f}"
    
    # Draw rectangle
    cv2.rectangle(image_np, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    
    # Put label
    cv2.putText(image_np, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the image (if running locally)
# display(IPImage("temp_output.jpg"))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Save the image with bounding boxes
cv2.imwrite("output_with_bboxes_coco.jpg", image_np)

True